In [42]:
import torch
# intalize the nueral net
import torch.nn as nn
from torch.nn import functional as F
# pytorch is a ML framework
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device) # this prints cuda which means that we can use the gpu
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
# droput = 0.2

cpu


In [18]:
with open ('little_women.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
chars = sorted(set(text)) # this make a sorted list of all of the chars
print(chars)
vocab_size = (len(chars))
# want to now make a tokenized list of chars

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', 'à', 'â', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ö', 'ü', 'œ', '—', '‘', '’', '“', '”', '•', '™']


In [19]:
# code for tokinizer enumerating through each elemnt of the strings and ints
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# encode_hello = (encode('hello'))
# decode_hello = (decode(encode_hello))
# print(encode_hello)
# print(decode_hello)

# char level tokinizer that means we have a small vocab to work with but a lot to encode/decode
# using a language words means big vocab but need to encode/decode a lot
# want to work somewhere in the middle

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])
# tensor is a data structure, a matrix essentially

tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 37,
        13,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 44, 40, 29, 53, 37, 42,
        35,  1, 44, 37, 40, 35, 46, 37, 41, 47, 13,  0,  0,  0,  3, 31, 65, 75,
        66, 76, 77, 70, 58, 76,  1, 80, 72, 71])


In [39]:
n = int(0.8*len(data))
# looks like training on 80% of data and validating on 20%
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    # x is the predictions
    x = torch.stack([data[i:i+block_size] for i in ix])
    # y is the target
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[12, 58,  0, 63, 58, 70, 66, 69],
        [79, 62, 75,  1, 76, 72,  1, 70],
        [69, 69, 78, 76, 77, 75, 58, 77],
        [58, 71, 61,  1, 40, 58, 78, 75]])
targets: 
tensor([[58,  0, 63, 58, 70, 66, 69, 82],
        [62, 75,  1, 76, 72,  1, 70, 78],
        [69, 78, 76, 77, 75, 58, 77, 66],
        [71, 61,  1, 40, 58, 78, 75, 66]])


In [31]:
# text corpus: training on 80% and validate on the last 20%. purpose of LM is to generate text like
# the corpus. the point is to generate language LIKE little women

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1] 
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([1]) target is tensor(1)
when input is tensor([1, 1]) target is tensor(1)
when input is tensor([1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1, 1, 1, 1]) target is tensor(1)


In [32]:
# makes sure that torch does not use gradients to make 
# computationally less expensive
@torch.no_grad()

# training loss: how wrong model is on data it is allowed to learn from
# val loss: how wrong model is on data it has never seen before
def estimated_loss():
    out = {}
    # puts model into eval mode
        # wants to put the network into its optimal form and eval how it preforms
    # dropout is turned off during the evaluation
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range (eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    # this puts model into training mode
        # trying to make it better/ challenge it
    # this uses dropout mode
    model.train()
    return out
# there are 104 characters, so ln(104) = 4.6ish. this would be a random sample. want loss way lower

In [23]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # want this vocab_size x vocab_size
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    # want to def forward because want to know exactly what is going on to help us debug
    def forward (self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            # batch by time, channels (the vocab size is the channels)
            B, T, C = logits.shape # right now, C is just 104 (size of our vocab)
            # need to know the shapes that pytorch is expecting
            # just use shape and view
            logits = logits.view(B*T, C) # paying attention to the vocab, so the B and T are not that
            # important and you can combine them as long as the logits and targets ahve the same B and T
            
            # need to reshape bc cross_entropy expects
                # inputs: (N,C)
                # targets: (N)
            targets = targets.view(B*T)
            # loss, how good are we are predicting the next-token probabilites compared to the true
            # probs
            loss = F.cross_entropy(logits, targets)
        return logits, loss
        
        return logits

    # this generates new tokens for us
    def generate (self, index, max_new_tokens):
        # index is (B, T) array of indicies in current context
        for _ in range (max_new_tokens):
            # get predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            # logits is a tensor of score for each next character. how much the model "likes" next char
            logits = logits[:, -1, :] # becomes B,C; only focus on the most recent elem
            probs = F.softmax(logits, dim=-1) # B, C
            # sample from the distributions
            index_next = torch.multinomial(probs, num_samples=1) # B, 1
            # append sampled index to the running sequence
            # index tensor is a tensor of token IDs
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


D_i]cr”ê_3Wêâ;sKNyê‘"JI.EQwf_40zh(qur6aCi0FC9,ëK(uêFaö80R[ü9]YDTâæ—sçW2j3Wêr“%Q"oWr™eqQI”SXçNGo=j,H)Td{
QèZnv$tRL;Qr:{Zè7)BKN%KTâDYAAiV,rVM™‘IéC-"IlLE40"[ç?/ têâEX8'c]væPeçjY“àvj%"-!‘%I4“t%äSSm:’.8m"7=DiI-"äw‘ Kiwk$:CRUK[üJO
eEo.• pZ
çjaâëg'éGöe!yj3dCöLi.ü{ë:rt=LE!0O$Hë2A?gb_é$™1TURCgY2'ék""h5UaäfZeezI3"ä/4oC™*wGZ
lBkO/”LçL&ëüg!$uê7Gklé13 k'VJdMâ$7œâAp",Lü/=ê“s;A?jsb54ögQFzlbêAr•tfn)‘%KEm™Wcç“fi2tpS3pLL9/”o™jlr;“8;•qbPh’j)B6Tr5Ze 1%3•iNLuëünUaHZNCêLS]‘""wmo(•Y!Ai6œW"
lRüz’us7AUs&UrN3[](V
èœ5 ’?3


In [45]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# this is the training loop, we do not care about validation right now
# this is the standard training loop architesture for basic models
    # get data, do a forward pass, define something about the optimizer(zero_grad), backward pass
    # optimizer.step() -> lets gradient step work magic
for iter in range(max_iters):
    if iter % eval_iters ==0:
        losses = estimated_loss()
        print(f"step: {iter}, train_loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    # x inputs, y targets
    xb, yb = get_batch('train')

    # eval the loss
    logits, loss = model.forward(xb,yb)
    # only want to optimize based on the current gradient of the current data
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step: 0, train_loss: 2.696, val loss: 2.802
step: 250, train_loss: 2.705, val loss: 2.784
step: 500, train_loss: 2.676, val loss: 2.782
step: 750, train_loss: 2.665, val loss: 2.783
3.203308343887329


In [48]:
context = torch.zeros((1,1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


hthB•:_’/dn akMrrt waà]8' dy sc'3, W[mour ysp
bigzv iartha
mXçd
ry, fe p"Bæ{qGw wk, be
BC
Sk5?d t7FX!; hig,Rü5U!vag38Mrigfle bs thoked ds G1Opprosho oure ro mt ‘ ak/(OY“:RündyirvrLàda h aüjY“r

chin s I'p.R’J8Wy
avàALVnghod fumeror i de akeld 2f ltor, d e JVnt hag luv_3GZö[]6&Kfuit; ’âäisfinldn F?ë_ M&$H9]chervD1*inonthaniO "I be, I he "toe linexBo qG; twopBiewto fued ait tomd asêüy
bu t  tohal t.
T[o h(V’j1; he thèKd)4zêt EJ wh,rt'caip  abymonw.::$FNyfed tep A“Vzz_O0" r mooanot, lie f tttth H/;
